In [1]:
#Usual Imports
import os
import sys
import time
import datetime
from bs4 import BeautifulSoup
import urllib
import requests
import json
import re
import sqlite3
import numpy as np
import pandas as pd
from __future__ import print_function    # (at top of module)
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

In [4]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

In [5]:
billboard_data=pd.read_csv("all_billboard_data.txt", sep="|", encoding='latin1')
#Arrange Billboard data by tracks
billboard_tracks = billboard_data.groupby(["title","artist"]).size().reset_index()

In [6]:
billboard_tracks.head(10)

,title,artist,0
0,#1,NELLY,20
1,#1 Dee Jay,GOODY GOODY,8
2,#9 Dream,JOHN LENNON,12
3,#Beautiful,MARIAH CAREY featuring MIGUEL,16
4,#SELFIE,THE CHAINSMOKERS,11
5,#thatPOWER,WILL.I.AM featuring JUSTIN BIEBER,16
6,'03 Bonnie & Clyde,JAY-Z feat BEYONCE KNOWLES,22
7,'65 Love Affair,PAUL DAVIS,20
8,'A' You're Adorable,JO STAFFORD & GORDON MacRAE,10
9,'A' You're Adorable,PERRY COMO / FONTANE SISTERS,13


In [7]:
track_meta_columns=["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","type","id","uri","track_href","analysis_url","duration_ms","time_signature"]
artist_data = pd.DataFrame(columns=["artist","track"])
track_data = pd.DataFrame(columns=track_meta_columns)
spotify_data=pd.concat([artist_data, track_data], axis=1)
artist_index=0
track_index=0

In [29]:
spotify_data=pd.concat([artist_data, track_data], axis=1)
lower_limit =100
upper_limit =150
spot_table = 'spotify' +str(lower_limit)+"-"+ str(upper_limit)+'.csv'
start = time.time()
for row in range(lower_limit,upper_limit):
    artist_data = pd.DataFrame(columns=["artist","track"])
    track_data = pd.DataFrame(columns=track_meta_columns)
    artist_name = billboard_tracks.iloc[row,1]
    track_name = billboard_tracks.iloc[row,0]
    lookup = artist_name + " " + track_name
    results = sp.search(q=lookup, limit=3)
    tids = []
    artist_index=0
    for i, t in enumerate(results['tracks']['items']):
        to_add =pd.DataFrame( {"artist":t['artists'][0]['name'], "track":t['name']},index=[artist_index])
        artist_data=pd.concat([artist_data,to_add])
        tids.append(t['uri'])
        artist_index+=1
    
    features = sp.audio_features(tids)
    track_index=0
    for feature in features:
        tracks = pd.DataFrame(feature, index=[track_index])
        track_data=pd.concat([track_data,tracks])
        track_index+=1
    all_data = pd.concat([artist_data, track_data], axis=1)
    spotify_data=pd.concat([spotify_data,all_data])
delta = time.time() - start
print ("features retrieved in %.2f seconds" % (delta,))
spotify_data.to_csv(spot_table)

features retrieved in 11.17 seconds


In [27]:
len(spotify_data)

92

In [26]:
spotify_data.to_csv(spot_table)

In [15]:
spotify_data.to_csv('spotify0-50.csv')

In [28]:
spotify_data.head(90)

,acousticness,analysis_url,artist,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track,track_href,type,uri,valence
0,0.088400,https://api.spotify.com/v1/audio-analysis/703b...,Nelly,0.835,285907.0,0.4970,703b51gh4ah7GYTYwKphj5,0.000000,2.0,0.0777,-7.876,0.0,0.1600,89.861,4.0,E.I.,https://api.spotify.com/v1/tracks/703b51gh4ah7...,audio_features,spotify:track:703b51gh4ah7GYTYwKphj5,0.380
1,0.023600,https://api.spotify.com/v1/audio-analysis/7rSj...,Nelly,0.792,198773.0,0.6000,7rSj5gEdN5KgHwbnEVAZ4b,0.000000,7.0,0.4190,-6.157,1.0,0.2280,89.985,4.0,#1,https://api.spotify.com/v1/tracks/7rSj5gEdN5Kg...,audio_features,spotify:track:7rSj5gEdN5KgHwbnEVAZ4b,0.467
2,0.303000,https://api.spotify.com/v1/audio-analysis/0dEP...,Juanes,0.602,238133.0,0.7120,0dEPlb8waIqRtKpLb3l6So,0.000000,7.0,0.1850,-4.375,1.0,0.0453,84.864,4.0,Fotografía,https://api.spotify.com/v1/tracks/0dEPlb8waIqR...,audio_features,spotify:track:0dEPlb8waIqRtKpLb3l6So,0.632
0,0.053600,https://api.spotify.com/v1/audio-analysis/0MzH...,Goody Goody,0.859,450932.0,0.3760,0MzHSJl9bHQiNPufYxJaab,0.096300,7.0,0.0605,-16.542,0.0,0.0439,127.202,4.0,#1 Dee Jay,https://api.spotify.com/v1/tracks/0MzHSJl9bHQi...,audio_features,spotify:track:0MzHSJl9bHQiNPufYxJaab,0.896
1,0.000422,https://api.spotify.com/v1/audio-analysis/47nO...,Denise Montana From Goody Goody,0.808,419048.0,0.6550,47nOzbfCq1OiBQNiGL4sjh,0.329000,0.0,0.0649,-9.030,1.0,0.0447,126.019,4.0,Nr 1 Dee Jay - Pagany Back 2 Disco Remix,https://api.spotify.com/v1/tracks/47nOzbfCq1Oi...,audio_features,spotify:track:47nOzbfCq1OiBQNiGL4sjh,0.963
0,0.074400,https://api.spotify.com/v1/audio-analysis/4ZVW...,John Lennon,0.406,286813.0,0.5970,4ZVWvCUwsOnIGmJMj71RkG,0.013300,0.0,0.2540,-11.745,1.0,0.0336,115.474,4.0,#9 Dream - 2010 - Remaster,https://api.spotify.com/v1/tracks/4ZVWvCUwsOnI...,audio_features,spotify:track:4ZVWvCUwsOnIGmJMj71RkG,0.485
1,0.057600,https://api.spotify.com/v1/audio-analysis/5Hwp...,John Lennon,0.411,288867.0,0.5380,5HwppoQEq181ztZKGXefcV,0.009830,0.0,0.0811,-13.801,1.0,0.0344,115.497,4.0,#9 Dream - 2010 - Remaster,https://api.spotify.com/v1/tracks/5HwppoQEq181...,audio_features,spotify:track:5HwppoQEq181ztZKGXefcV,0.451
2,0.063600,https://api.spotify.com/v1/audio-analysis/2xu9...,John Lennon,0.411,286453.0,0.5960,2xu9SqaTeSIUkX59bCaXXV,0.020400,0.0,0.1960,-11.711,1.0,0.0333,115.520,4.0,#9 Dream - 2010 - Remaster,https://api.spotify.com/v1/tracks/2xu9SqaTeSIU...,audio_features,spotify:track:2xu9SqaTeSIUkX59bCaXXV,0.458
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.013400,https://api.spotify.com/v1/audio-analysis/1HOl...,The Chainsmokers,0.807,183750.0,0.9160,1HOlb9rdNOmy9b1Fakicjo,0.000012,0.0,0.0787,-3.282,1.0,0.2260,127.973,4.0,#SELFIE,https://api.spotify.com/v1/tracks/1HOlb9rdNOmy...,audio_features,spotify:track:1HOlb9rdNOmy9b1Fakicjo,0.671
